In [1]:
import pandas as pd
import pyarrow.parquet as pq
import os
import numpy as np

### Data Load

In [2]:
current_path = os.getcwd()
print(current_path)

/Users/hyemi/Desktop/FairFares


In [3]:
path = os.path.join(current_path, 'Data/fhvhv_tripdata_2023-03.parquet')
df_org = pq.read_table(source=path).to_pandas()

In [4]:
path_yellow = os.path.join(current_path, 'Data/yellow_tripdata_2023-03.parquet')
df_yellow_org = pq.read_table(source=path_yellow).to_pandas()

In [5]:
path_green = os.path.join(current_path, 'Data/green_tripdata_2023-03.parquet')
df_green_org = pq.read_table(source=path_green).to_pandas()

### Taxi zones

In [6]:
import csv
csv_path = os.path.join(current_path, 'Data/taxi_zone_lookup.csv')

manhattan_zone = []
with open(csv_path, newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in spamreader:
        if 'LocationID' == row[0]:
            continue
        zone, borough = row[0], row[1]
        if borough == 'Manhattan' and int(zone) not in [103, 104, 105, 153, 194, 202]:
            manhattan_zone.append(int(zone))
print(len(manhattan_zone))

63


In [7]:
# loc_1 = [128, 127, 243, 244, 120]
# loc_2 = [116, 152, 166]
# loc_3 = [42, 41, 74, 75]
# loc_4 = [24, 151, 238, 239, 143, 142]
# loc_5 = [43]
# loc_6 = [236, 263, 262, 237, 141, 140]
# loc_7 = [50, 48, 246, 68]
# loc_8 = [163, 230, 161, 162, 100, 186, 164, 90, 234]
# loc_9 = [229, 233, 170, 137, 107, 224]
# loc_10 = [158, 249, 113, 114, 125, 211, 144]
# loc_11 = [79, 4, 148, 232]
# loc_12 = [231, 45, 209, 13, 261, 87, 88, 12]
# loc = {1: loc_1, 2: loc_2, 3: loc_3, 4: loc_4, 5: loc_5, 6: loc_6, 7: loc_7, 8: loc_8, 9: loc_9, 10: loc_10, 11: loc_11, 12: loc_12}

loc_1 = [128, 127, 243, 244, 120, 116, 152, 166]
loc_2 = [42, 41, 74, 75]
loc_3 = [24, 151, 238, 239, 143, 142, 43]
loc_4 = [236, 263, 262, 237, 141, 140]
loc_5 = [50, 48, 246, 68]
loc_6 = [163, 230, 161, 162, 100, 186, 164, 90, 234]
loc_7 = [229, 233, 170, 137, 107, 224]
loc_8 = [158, 249, 113, 114, 125, 211, 144]
loc_9 = [79, 4, 148, 232]
loc_10 = [231, 45, 209, 13, 261, 87, 88, 12]
loc = {1: loc_1, 2: loc_2, 3: loc_3, 4: loc_4, 5: loc_5, 6: loc_6, 7: loc_7, 8: loc_8, 9: loc_9, 10: loc_10}

all_zones = []
several_zones = []
for i in range(1, len(loc.values())+1):
    all_zones += loc[i]
    several_zones.append(loc[i])
print('zones in data = defined zones ?', sorted(all_zones) == sorted(manhattan_zone))

zones in data = defined zones ? True


In [8]:
print(len(all_zones))

63


In [9]:
print(len(df_org.index))
df = df_org.copy()

20413539


### Timeframe
* time_frame: "2023-03-01 18:00 - 20:00"

In [11]:
time_start = "2023-03-03 18:00:00"
time_end = "2023-03-03 20:00:00"

###  Data Preprocessing

In [12]:
df_yellow = df_yellow_org.copy()
df_green = df_green_org.copy()
df_platform = df_org.copy()

In [13]:
df_platform.columns

Index(['hvfhs_license_num', 'dispatching_base_num', 'originating_base_num',
       'request_datetime', 'on_scene_datetime', 'pickup_datetime',
       'dropoff_datetime', 'PULocationID', 'DOLocationID', 'trip_miles',
       'trip_time', 'base_passenger_fare', 'tolls', 'bcf', 'sales_tax',
       'congestion_surcharge', 'airport_fee', 'tips', 'driver_pay',
       'shared_request_flag', 'shared_match_flag', 'access_a_ride_flag',
       'wav_request_flag', 'wav_match_flag'],
      dtype='object')

In [14]:
print("yellow cab", df_yellow.columns)
print("green cab", df_green.columns)
print("platforms", df_platform.columns)

yellow cab Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'Airport_fee'],
      dtype='object')
green cab Index(['VendorID', 'lpep_pickup_datetime', 'lpep_dropoff_datetime',
       'store_and_fwd_flag', 'RatecodeID', 'PULocationID', 'DOLocationID',
       'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax',
       'tip_amount', 'tolls_amount', 'ehail_fee', 'improvement_surcharge',
       'total_amount', 'payment_type', 'trip_type', 'congestion_surcharge'],
      dtype='object')
platforms Index(['hvfhs_license_num', 'dispatching_base_num', 'originating_base_num',
       'request_datetime', 'on_scene_datetime', 'pickup_datetime',
       'dropoff_datetime', 'PULocationID', 'DOL

In [15]:
def cab_filter(df, time_start, time_end, all_zones, several_zones, taxi_type='yellow', N=13):
    print("%-30s %-4s" % ("taxi type", taxi_type))
    
    print("%-30s %4d" % ("original length", len(df)))
    # time filter
    if taxi_type == 'yellow':
        mask = (df.tpep_pickup_datetime >= time_start) & (df.tpep_pickup_datetime < time_end)
    elif taxi_type == 'green':
        mask = (df.lpep_pickup_datetime >= time_start) & (df.lpep_pickup_datetime < time_end)
    else:
        df.drop(df.loc[df['shared_match_flag']=='Y'].index, inplace=True)
        mask = (df.pickup_datetime >= time_start) & (df.pickup_datetime < time_end)
    df = df[mask]
    #print("%-30s %4d" % ("after time filter", len(df)))
    
    # make closed network
    df = df[df.PULocationID.isin(all_zones)]
    df = df[df.DOLocationID.isin(all_zones)]
    #print("%-30s %4d" % ("after closed network", len(df)))
    
    # remove the loop
    for zone in several_zones:
        loop = df['PULocationID'].isin(zone) & df['DOLocationID'].isin(zone)
        df = df[~loop]
    #print("%-30s %4d" % ("after removing the loop", len(df)))
    
    # naming zone
    for key, value in loc.items():
        df.loc[df['PULocationID'].isin(value), 'PU_loc'] = key
        df.loc[df['DOLocationID'].isin(value), 'DO_loc'] = key
    
    ### =============================================================================================== ###
    
    if taxi_type == 'yellow':
        t_do = 'tpep_dropoff_datetime'
        t_pu = 'tpep_pickup_datetime'
        dist = 'trip_distance'
    elif taxi_type == 'green':
        t_do = 'lpep_dropoff_datetime'
        t_pu = 'lpep_pickup_datetime'
        dist = 'trip_distance'
    else:
        t_do = 'dropoff_datetime'
        t_pu = 'pickup_datetime'
        dist = 'trip_miles'
        df['total_amount'] = df['base_passenger_fare'] + df['congestion_surcharge'] + df['tolls'] +\
        df['sales_tax'] + df['airport_fee'] + df['tips']
    
    # Eliminate the row with negative fare
    df = df[df['total_amount'] > 0]
    print("%-30s %4d" % ("# of trips", len(df)))
    
    df['trip_time'] = (df[t_do] - df[t_pu])/np.timedelta64(1, 'm')
    df['price_per_miles'] = df['total_amount']/df[dist]
    df['price_per_time'] = df['total_amount']/df['trip_time']
    
    if taxi_type == 'yellow' or taxi_type == 'green':
        num_of_trips = pd.DataFrame(0, index=range(1, N+1), columns=range(1, N+1))
        sum_price = pd.DataFrame(index=range(1, N+1), columns=range(1, N+1))
        max_price = pd.DataFrame(index=range(1, N+1), columns=range(1, N+1))
        min_price = pd.DataFrame(index=range(1, N+1), columns=range(1, N+1))
        sum_dist = pd.DataFrame(index=range(1, N+1), columns=range(1, N+1))
        sum_time = pd.DataFrame(index=range(1, N+1), columns=range(1, N+1))
        # Calculate average price/miles & price/time for each pair of locations
        for i in range(1, N+1):
            for j in range(1, N+1):
                trips = df[(df['PU_loc'] == i) & (df['DO_loc'] == j)]
                if len(trips) > 0:
                    num_of_trips.loc[i, j] = len(trips)
                    sum_price.loc[i, j] = sum(trips['total_amount'])/len(trips)
                    max_price.loc[i, j] = max(trips['total_amount'])
                    min_price.loc[i, j] = min(trips['total_amount'])
                    sum_dist.loc[i,j] = sum(trips[dist])
                    sum_time.loc[i,j] = sum(trips['trip_time'])
                else:
                    num_of_trips.loc[i, j] = 0
                    sum_price.loc[i, j] = 0
                    max_price.loc[i, j] = 0
                    min_price.loc[i, j] = 0
                    sum_dist.loc[i,j] = 0
                    sum_time.loc[i,j] = 0

        # Display the resulting table
        print("number of trips:")
        print(num_of_trips)
        
        return {'num': num_of_trips, 'sum_p': sum_price, 'p_max': max_price, 'p_min': min_price,\
                'sum_dist': sum_dist, 'sum_time': sum_time}
    else:
        _dict = []
        for hvfhs_license_num in ['HV0003', 'HV0005']:
            num_of_trips = pd.DataFrame(0, index=range(1, N+1), columns=range(1, N+1))
            sum_price = pd.DataFrame(index=range(1, N+1), columns=range(1, N+1))
            max_price = pd.DataFrame(index=range(1, N+1), columns=range(1, N+1))
            min_price = pd.DataFrame(index=range(1, N+1), columns=range(1, N+1))
            sum_dist = pd.DataFrame(index=range(1, N+1), columns=range(1, N+1))
            sum_time = pd.DataFrame(index=range(1, N+1), columns=range(1, N+1))

            for i in range(1, N+1):
                for j in range(1, N+1):
                    trips = df[(df['PU_loc'] == i) & (df['DO_loc'] == j) & (df['hvfhs_license_num'] == hvfhs_license_num)]
                    if len(trips) > 0:
                        num_of_trips.loc[i, j] = len(trips)
                        sum_price.loc[i, j] = sum(trips['total_amount'])
                        max_price.loc[i, j] = max(trips['total_amount'])
                        min_price.loc[i, j] = min(trips['total_amount'])
                        sum_dist.loc[i,j] = sum(trips[dist])
                        sum_time.loc[i,j] = sum(trips['trip_time'])
                    else:
                        num_of_trips.loc[i, j] = 0
                        sum_price.loc[i, j] = 0
                        max_price.loc[i, j] = 0
                        min_price.loc[i, j] = 0
                        sum_dist.loc[i,j] = 0
                        sum_time.loc[i,j] = 0
            
            # Display the resulting table
            output = 'Uber' if hvfhs_license_num == 'HV0003' else 'Lyft'
            print("\n=== %s ===" %output)
            print("number of trips:")
            print(num_of_trips)
            
            _dict.append({'num': num_of_trips, 'sum_p': sum_price, 'p_max': max_price, 'p_min': min_price,\
                          'sum_dist': sum_dist, 'sum_time': sum_time})
              
        return _dict

In [16]:
[uber, lyft] = cab_filter(df_platform, time_start, time_end, all_zones, several_zones, taxi_type='platforms', N=len(loc.values()))

taxi type                      platforms
original length                20413539
# of trips                     26581

=== Uber ===
number of trips:
     1    2    3    4    5    6    7    8    9    10
1     0  232  147   63   63   98   25   44   29   14
2   251    0  141  132   45   92   36   34   41   20
3   131  141    0  388  187  463  111  152   80   56
4    53  133  389    0  142  542  252  281  139  104
5    51   35  197  125    0  754  237  450  179  204
6    77   76  438  474  717    0  654  887  435  297
7    20   34  144  277  229  761    0  574  408  176
8    18   26   98  138  314  705  273    0  527  463
9    13   16   43   71  150  313  201  549    0  192
10   11   22   64   91  187  351  150  674  344    0

=== Lyft ===
number of trips:
    1    2    3    4    5    6    7    8    9    10
1    0  130   63   31   31   34   11    5   15    4
2   98    0   75   80   26   28   13    5   17    9
3   67   50    0  105   60  124   29   38   35   18
4   14   60  131    0   47  1

In [17]:
print(uber['sum_p'])

         1        2         3        4         5         6        7   \
1         0   3739.5   3572.85  2001.97   2011.55   3705.52  1054.85   
2   4407.62        0    2934.5  2778.13   1540.12   3351.63  1256.98   
3   2842.22  2792.57         0  8268.01   4074.44  11819.52  3285.24   
4   1637.25  2634.12    8837.2        0    4388.2  15140.43   5533.9   
5   1815.11  1196.47   4469.59  4052.71         0  15134.07  5703.54   
6   3022.69  2472.38  11574.04  11934.5  15375.17         0  12732.0   
7    776.45  1024.75   4292.98   6206.3   5689.66   15329.2        0   
8    721.11  1059.72   3683.34  5594.73   7618.18  17697.82  6503.41   
9    605.95   574.92   1683.88  2318.87    4620.1   8686.08  3694.83   
10   510.07   911.19    2571.9  4272.96   5552.14  12096.18  4598.87   

          8         9         10  
1     1981.8   1360.53     748.3  
2    1519.51   1538.95    819.43  
3    5896.75   3295.85   2400.99  
4   11390.66   4472.57   4545.89  
5   10698.96   5583.67   6384.86

In [18]:
yellow = cab_filter(df_yellow, time_start, time_end, all_zones, several_zones, taxi_type='yellow', N=len(loc.values()))

taxi type                      yellow
original length                3403766
# of trips                     9692
number of trips:
    1    2    3    4    5    6    7    8    9    10
1    0   18   33   15    2    5    2    1    4    0
2   12    0   36   45    2    8   10    1    4    1
3   94   82    0  353  152  275   91   53   31   19
4   21  132  390    0   86  387  257  101   73   31
5   10   11  140   79    0  302   86  159   41   35
6   22   55  457  632  453    0  587  465  220  153
7    9   18  112  286   97  376    0  206  191   47
8    6    6   44   56  114  232  107    0  164  128
9    3    3   16   33   33   70   59   80    0   24
10   6    4   21   43   44   72   32  137   79    0


In [19]:
green = cab_filter(df_green, time_start, time_end, all_zones, several_zones, taxi_type='green', N=len(loc.values()))

taxi type                      green
original length                72044
# of trips                      136
number of trips:
    1   2   3   4   5   6   7   8   9   10
1    0   8  14   3   4   3   1   1   2   0
2   11   0  34  32   3   1   1   1   0   0
3    0   3   0   7   1   3   2   0   0   0
4    0   0   1   0   0   0   0   0   0   0
5    0   0   0   0   0   0   0   0   0   0
6    0   0   0   0   0   0   0   0   0   0
7    0   0   0   0   0   0   0   0   0   0
8    0   0   0   0   0   0   0   0   0   0
9    0   0   0   0   0   0   0   0   0   0
10   0   0   0   0   0   0   0   0   0   0


In [20]:
N = len(loc.values())
total_trips = pd.DataFrame(0, index=range(1, N+1), columns=range(1, N+1))
total_trips = total_trips.add(uber['num'])
total_trips = total_trips.add(lyft['num'])
total_trips = total_trips.add(yellow['num'])
total_trips = total_trips.add(green['num'])
print(total_trips)

min_price = pd.DataFrame(index=range(1, N+1), columns=range(1, N+1))
max_price = pd.DataFrame(index=range(1, N+1), columns=range(1, N+1))
avg_price = pd.DataFrame(index=range(1, N+1), columns=range(1, N+1))
avg_time = pd.DataFrame(index=range(1, N+1), columns=range(1, N+1))
avg_dist = pd.DataFrame(index=range(1, N+1), columns=range(1, N+1))

for i in range(1, N+1):
    for j in range(1, N+1):
        lst_dist = [uber['sum_dist'].loc[i,j], lyft['sum_dist'].loc[i,j], yellow['sum_dist'].loc[i,j], green['sum_dist'].loc[i,j]]
        lst_time = [uber['sum_time'].loc[i,j], lyft['sum_time'].loc[i,j], yellow['sum_time'].loc[i,j], green['sum_time'].loc[i,j]]
        lst_num = [uber['num'].loc[i,j], lyft['num'].loc[i,j], yellow['num'].loc[i,j], green['num'].loc[i,j]]
        avg_dist.loc[i,j] = sum(lst_dist)/sum(lst_num) if i != j else 0
        avg_time.loc[i,j] = sum(lst_time)/sum(lst_num) if i != j else 0
        if i != j:
            lst = [uber['p_min'].loc[i,j], lyft['p_min'].loc[i,j], yellow['p_min'].loc[i,j], green['p_min'].loc[i,j]]
            min_price.loc[i,j] = min([x for x in lst if x != 0])
            lst = [uber['p_max'].loc[i,j], lyft['p_max'].loc[i,j], yellow['p_max'].loc[i,j], green['p_max'].loc[i,j]]
            max_price.loc[i,j] = min([x for x in lst if x != 0])
            lst_price =[uber['sum_p'].loc[i,j], lyft['sum_p'].loc[i,j], yellow['sum_p'].loc[i,j], green['sum_p'].loc[i,j]]
            avg_price.loc[i,j] = sum(lst_price)/sum(lst_num)

     1    2    3     4     5     6     7     8    9    10
1     0  388  257   112   100   140    39    51   50   18
2   372    0  286   289    76   129    60    41   62   30
3   292  276    0   853   400   865   233   243  146   93
4    88  325  911     0   275  1084   622   447  281  162
5    77   65  414   259     0  1289   395   731  291  283
6   126  167  990  1229  1354     0  1394  1549  795  511
7    38   66  291   661   415  1341     0   993  778  296
8    33   40  176   242   542  1107   463     0  895  700
9    22   27   70   129   232   490   351   806    0  298
10   24   32  100   179   290   518   225   976  548    0


In [21]:
print(min_price)

       1      2      3      4      5      6      7      8      9      10
1     NaN    5.2   5.66  16.61  13.63  22.18  27.29  22.54  34.38  34.05
2    5.83    NaN   6.71    7.5   21.4   12.9  15.63  27.56  21.12  30.23
3    5.14   7.73    NaN   5.82   6.85   8.88   12.3  19.44  21.43  24.25
4   17.96   7.58   8.57    NaN  11.45    8.0   9.67   17.2  13.62   6.98
5   16.61  19.34   9.44  11.74    NaN   7.21    9.8   7.78  15.42  14.41
6   20.26  14.74   8.17   8.68   7.99    NaN   4.71   7.78  10.77  16.15
7   26.92   17.7  17.23   7.85  12.59   7.68    NaN   5.57   9.24  17.57
8    27.7   23.7   20.5  19.84   9.67   8.15   6.78    NaN   2.07   0.53
9    37.0  22.58  22.38  15.66  15.56   9.09   8.75   5.07    NaN   9.55
10  36.83  28.58  25.04  21.44    9.2  11.74  10.75   5.97   9.39    NaN


In [22]:
print(max_price)

       1      2      3      4      5      6      7      8      9      10
1     NaN  21.72  27.23  29.34   37.4  44.16  47.44  38.45  60.12  46.78
2    28.4    NaN  33.55  30.72   36.6  29.34  31.44   43.2   45.5   47.9
3    37.1   11.4    NaN  29.97  28.92  38.15   40.8   53.3   49.2  46.94
4   51.54   36.2   21.9    NaN  39.83  42.48  34.48  48.92   44.2  45.42
5    50.2  46.58  35.97  42.48    NaN  37.24   36.6  38.28  40.43  47.52
6   46.37  46.02  52.56  49.03   48.7    NaN  33.15  45.84  47.75  50.88
7   53.99  34.06  44.69  30.74   36.9   34.3    NaN  39.95  43.29  41.56
8   50.38  45.82  43.19  51.72   46.0  39.96  29.17    NaN  47.04   32.0
9   57.17  41.33  45.81  36.76   45.0   49.2  29.72  29.22    NaN  28.35
10   56.0  41.83  58.56   62.6  40.73  50.88  44.16  37.94  39.12    NaN


In [23]:
print(avg_price.applymap(lambda x: round(x, 1)))

      1     2     3     4     5     6     7     8     9     10
1    NaN  14.7  19.0  26.6  29.9  36.3  40.3  44.0  44.3  51.3
2   16.0   NaN  15.3  14.6  32.0  33.6  29.4  44.6  36.6  41.4
3   14.3  13.6   NaN  11.8  13.1  17.0  17.9  29.5  31.4  33.0
4   24.2  11.6  12.3   NaN  20.9  17.6  12.5  30.3  23.1  33.8
5   30.3  28.5  14.1  21.4   NaN  14.7  18.3  18.1  25.8  26.6
6   31.8  21.3  13.9  11.9  13.9   NaN  11.0  16.7  19.0  24.7
7   31.0  21.5  18.1  12.2  18.8  14.3   NaN  18.5  15.4  25.1
8   36.0  34.9  26.8  29.5  18.7  19.2  17.4   NaN  14.9  15.6
9   43.6  33.1  30.1  23.3  26.1  22.8  15.0  15.6   NaN  18.4
10  36.9  37.1  31.6  32.2  24.0  28.6  25.1  16.7  17.9   NaN


In [24]:
print(avg_dist)

           1         2         3         4         5         6         7   \
1           0  2.414196  3.367992  4.476054   6.35225   6.82235  8.039769   
2    2.439449         0  2.412993  2.147509  5.829118  5.339938  5.147933   
3    3.423918  2.523011         0  1.952321  2.212232  2.312779  3.193356   
4    5.034648  2.219714  1.936137         0  3.411167  2.346753  2.235019   
5    6.895727  6.143708  2.275365  3.552274         0  1.466987  1.993339   
6    7.288429  4.841024  2.357212  2.183819  1.555332         0  1.375315   
7    7.491395  4.800424   3.28823  2.090256  2.056014  1.414358         0   
8    9.701364  7.458275  4.674972  4.748145  2.362596  2.099642  2.131963   
9    9.906773  7.963926  5.692514  4.471047  3.459095  2.571635  1.868097   
10  10.367708  8.963125   6.05198  6.961855     3.571  4.122633  4.692156   

          8         9          10  
1   8.927569  10.05962  10.482667  
2    7.96522   7.91271     9.3142  
3   4.712062  5.892445   6.445215  
4   5.08

In [27]:
beta_df = pd.DataFrame(index=range(1, N+1), columns=range(1, N+1))
k_df = pd.DataFrame(index=range(1, N+1), columns=range(1, N+1))

for i in range(1, N+1):
    for j in range(1, N+1):
        if i != j:
            print(f"from {i} to {j}")
            p_u = uber['sum_p'].loc[i,j]/uber['num'].loc[i,j]
            q_u = uber['num'].loc[i,j]/total_trips.loc[i,j]
            p_max = max_price.loc[i,j]
            p_min = min_price.loc[i,j]#uber['p_min'].loc[i,j]#min_price.loc[i,j]
            q_min = 0.95

            beta = 1/(p_u-p_min) * np.log((1-q_u)/q_u * q_min/(1-q_min))
            k = q_u * np.exp(beta * p_u)/(1-q_u)

            # Store the optimized parameters in the respective DataFrames
            beta_df.loc[i, j] = beta
            k_df.loc[i, j] = k


            print("Expected Valuation: ", 1/beta * np.log(1+k))

# Print the resulting DataFrames
print("beta DataFrame:")
print(beta_df)

print("k DataFrame:")
print(k_df)

from 1 to 2
Expected Valuation:  17.886047241727205
from 1 to 3
Expected Valuation:  26.504961002957472
from 1 to 4
Expected Valuation:  33.20817320806205
from 1 to 5
Expected Valuation:  36.032745542455025
from 1 to 6
Expected Valuation:  44.1469044783403
from 1 to 7
Expected Valuation:  45.85291393009225
from 1 to 8
Expected Valuation:  82.78468239393393
from 1 to 9
Expected Valuation:  48.45827615936707
from 1 to 10
Expected Valuation:  67.84753149430266
from 2 to 1
Expected Valuation:  21.51670592652636
from 2 to 3
Expected Valuation:  20.739654663097348
from 2 to 4
Expected Valuation:  20.3333721407924
from 2 to 5
Expected Valuation:  36.08693955977944
from 2 to 6
Expected Valuation:  47.16862947305732
from 2 to 7
Expected Valuation:  38.046938165080576
from 2 to 8
Expected Valuation:  64.61511464162251
from 2 to 9
Expected Valuation:  42.382394546880576
from 2 to 10
Expected Valuation:  44.27913074715124
from 3 to 1
Expected Valuation:  20.71571748720468
from 3 to 2
Expected Valu

In [28]:
data = {'market_size': total_trips, 'beta': beta_df, 'k': k_df, 'avg_dist': avg_dist, 'avg_time': avg_time, 'uber': uber}

In [123]:
for key in data.keys():
    print(data[key])

     1    2    3     4     5     6     7     8    9    10
1     0  388  257   112   100   140    39    51   50   18
2   372    0  286   289    76   129    60    41   62   30
3   292  276    0   853   400   865   233   243  146   93
4    88  325  911     0   275  1084   622   447  281  162
5    77   65  414   259     0  1289   395   731  291  283
6   126  167  990  1229  1354     0  1394  1549  795  511
7    38   66  291   661   415  1341     0   993  778  296
8    33   40  176   242   542  1107   463     0  895  700
9    22   27   70   129   232   490   351   806    0  298
10   24   32  100   179   290   518   225   976  548    0
          1         2         3         4         5         6         7   \
1        NaN  0.233324  0.168591  0.198355  0.173549  0.136342  0.176018   
2   0.204878       NaN  0.227735  0.246737  0.237359  0.133517  0.131648   
3   0.239842  0.260515       NaN  0.274663   0.23672  0.168377   0.18751   
4   0.195607  0.299544  0.260379       NaN  0.179679  0.17

In [29]:
import pickle

with open('data_dictionary_logistic.pkl', 'wb') as f:
    pickle.dump(data, f)